# 🧠 AI Agents Bootcamp: Multi-Agent Bidding with CrewAI

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/vipbasil/aibootcamp/blob/main/Day%20II/Multi_Agent_Bidding.ipynb)

This notebook is part of the **AI Agents Bootcamp** (23–27 June 2025).  
It demonstrates how to:

- 🕸️ Build a Multi-Agent System (MAS) using CrewAI
- 🗂️ Define multiple agents with distinct roles and models
- ⚖️ Implement bidding logic for dynamic task allocation
- 🧠 Use locally hosted LLMs (via Ollama + DeepSeek)
- 📋 Coordinate tasks and evaluate outcomes within a MAS pipeline

## ⚙️ Step 1: Environment Setup
This installs and runs the `ollama` backend and exposes the service via localtunnel tunnel. Make sure to:
- Restart the runtime if needed
- Use the ngrok alternative (if Cloudflare is blocked or throttled)

In [ ]:
%pip install ollama
%pip install colab-xterm

## 🛠️ System Info Tools (Optional)

Installs utilities (`pciutils`, `lshw`) to inspect hardware specs — useful for checking GPU/CPU availability in Colab or local runtime.








In [ ]:
!sudo apt-get update
!sudo apt-get install pciutils lshw

## 📦 Ollama Installation

Downloads and installs Ollama via the official shell script — run this once per environment setup.

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

## 🔧 Step 2: Programmatic Model Management and Server Initialization

In this section, we:
- Import the required libraries for managing subprocesses, HTTP requests, and multithreading
- Start the Ollama server programmatically using a background thread
- Pull the required models (`deepseek-r1:7b`, `llama3`) using `ollama pull`
- Optionally include fallback to a smaller model (`deepseek-r1:1.5b`)
- Confirm the list of available models and test that the local Ollama server is running at `localhost:11434`

📌 **Why it matters**: This sets up your local model infrastructure for agent interaction. You'll later reference `localhost:11434` in your agent definitions to connect to these models.


In [ ]:
# Import necessary libraries
import subprocess
import requests
import json
import threading
from pprint import pprint

##  Launching the Ollama Server in Background

Before using any model, we need to start the **Ollama inference server**, which listens by default on `localhost:11434`.

This snippet:
- Defines a Python function `run_ollama()` that launches `ollama serve`
- Starts it in a **background thread**, so the notebook remains interactive
- Allows the server to stay active without blocking further cells

🛠️ **Note**: You only need to run this once per session. If you restart your Colab, re-run this cell before using any models.


In [ ]:
# Start the Ollama server
def run_ollama():
  subprocess.Popen(["ollama", "serve"])
thread = threading.Thread(target=run_ollama)
thread.start()

## 📥 Pulling Models

We download pre-trained models from the Ollama registry:
- `deepseek-r1:7b` – reasoning & code
- `llama3` – general-purpose assistant

In [ ]:
# Download the deepseek-r1:7b distilled model
!ollama pull deepseek-r1:7b
!ollama pull llama3
!ollama pull deepseek-coder:6.7b
# If this doesn't work, you can uncomment the below code to download a smaller model- deepseek-r1:1.5b
# !ollama pull deepseek-r1:1.5b

## 🪶 Pulling Lightweight SLMs

These small models are ideal for fast local agents and low-resource environments:
- `phi3:mini`, `tinyllama` – ultra-small general models
- `gemma:2b` – Google's compact chat model
- `deepseek-r1:1.5b` – distilled reasoning model

In [ ]:
!ollama pull phi3:mini
!ollama pull tinyllama
!ollama pull gemma:2b
!ollama pull deepseek-r1:1.5b

## 🔌 Test Ollama Server

Sends a test request to verify the Ollama server is running on `localhost:11434`.

In [ ]:
!curl http://127.0.0.1:11434

## 📄 Check Installed Models

Lists all models currently downloaded and available in your local Ollama environment.

In [ ]:
!ollama list

# 🧠 Starting the CrewAI Section

##Now we define agents using CrewAI, connected to our locally running Ollama models.  
##This enables multi-agent workflows powered by lightweight, self-hosted LLMs.


In [ ]:
# @title 👨‍🦯 Run this cell to hide all warnings (optional)
# Warning control
import warnings
warnings.filterwarnings('ignore')

# To avoid the restart session warning in Colab, exclude the PIL and
# pydevd_plugins packages from being imported. This is fine because
# we didn't execute the code in the kernel session afterward.

# import sys
# sys.modules.pop('PIL', None)

## ⚙️ Install Project Dependencies

This cell installs the required Python packages to run the multi-agent system with CrewAI and LangChain integrations.  
It includes:

- `crewAI`: the main framework for defining agents and task flows  
- `crewai_tools`: additional utilities to extend agent capabilities  
- `langchain_*`: integrations for Groq, Anthropic, and other backends  
- `cohere`: optional LLM support via Cohere API  
- `--quiet`: suppresses output for a cleaner notebook  
- `pip show`: verifies that all packages are successfully installed


In [ ]:
# @title ⬇️ Install project dependencies by running this cell
%pip install git+https://github.com/joaomdmoura/crewAI.git --quiet
%pip install crewai_tools langchain_groq langchain_anthropic langchain_community cohere --quiet
print("---")
%pip show crewAI crewai_tools langchain_groq langchain_anthropic langchain_community cohere

## 🧩 Step 3: CrewAI Integration


In [ ]:
# imports

from crewai import Agent, Task, Crew, Process
from textwrap import dedent
from crewai import LLM



## Define Agents
In CrewAI, agents are autonomous entities designed to perform specific roles and achieve particular goals. Each agent uses a language model (LLM) and may have specialized tools to help execute tasks.

## ⚙️ Configure the LLM Interface

This line initializes the LLM wrapper used by CrewAI to route all language model calls to a local Ollama instance.  

Key components:
- `model="ollama/deepseek-r1:7b"`: Specifies the local model name to use (e.g. `deepseek-r1:7b`)
- `base_url="http://127.0.0.1:11434"`: Connects to the locally running Ollama server (default port)

```python
llm = LLM(model="ollama/deepseek-r1:7b", base_url="http://127.0.0.1:11434")
```


In [ ]:


llm = LLM(model="ollama/deepseek-r1:7b", base_url="http://127.0.0.1:11434")


## ⚙️ Define Agent Specifications

This dictionary defines the configuration for each agent in your multi-agent system (MAS). Each agent includes:

- `role`: the functional label of the agent
- `goal`: the objective it should pursue
- `backstory`: a narrative that helps guide agent reasoning (contextual prompt)
- `llm`: the local model the agent will use (via Ollama)

Three agents are specified:

1. **Planner** — responsible for system-level planning  
2. **Coder** — writes and tests backend code  
3. **Reviewer** — checks outputs for errors and logic bugs

In [ ]:
agent_specs = {
    "Planner": {
        "role": "Planner",
        "goal": "Create structured development plans.",
        "backstory": "Expert in designing system workflows.",
        "llm" : "ollama/deepseek-r1:7b"
    },
    "Coder": {
        "role": "Coder",
        "goal": "Write and test code effectively.",
        "backstory": "Backend developer with API focus.",
        "llm" : "ollama/deepseek-coder:6.7b"
    },
    "Reviewer": {
        "role": "Reviewer",
        "goal": "Review outputs and catch issues.",
        "backstory": "Critical code reviewer and tester.",
        "llm" : "ollama/deepseek-r1:7b"
    }
}


## ⚙️ Instantiate Agents from Specification

This block dynamically creates agent instances from the `agent_specs` dictionary using a `for` loop.

- Each agent is initialized with its role, goal, backstory, and LLM model.
- All agents are added to the `_agents` list, which is later passed to the `Crew` object.
- Their names are stored in `_agent_names` for easy reference or mapping.
- The LLM is configured per agent using the local Ollama server and the agent-specific model.


In [ ]:

_agents = []
_agent_names = []
for name, spec in agent_specs.items():
    _agent_names.append(name)
    _agents.append( Agent(
        role=spec["role"],
        goal=spec["goal"],
        backstory=spec["backstory"],
        verbose=True,
        llm=LLM(model=spec["llm"], base_url="http://127.0.0.1:11434")
    ))



## ⚙️ Define Task List

This block prepares the task data to be assigned to agents.  
Each task includes:

- `description`: a short description of the task objective  
- `type`: used to match the task with a suitable agent role (e.g., `plan`, `code`, `review`)  
- `complexity`: an integer from 1–5 indicating task difficulty (used for bidding or scoring)

You can later convert these tasks into `Task` objects and assign them dynamically.


In [ ]:
# ✅ 2. Define tasks
from crewai import Task
from textwrap import dedent

task_data = [
    {"description": "Design the user login flow", "type": "plan", "complexity": 2},
    {"description": "Write the login API using FastAPI", "type": "code", "complexity": 3},
    {"description": "Review the login module for bugs", "type": "review", "complexity": 1},
    {"description": "Draft a database schema for user roles", "type": "plan", "complexity": 3},
    {"description": "Implement user roles in backend", "type": "code", "complexity": 4}
]

## ⚙️ Define LLM Response Function

This function sends a prompt to the local Ollama API endpoint and retrieves a text completion using a specified model (`deepseek-r1:7b`). It acts as a bridge between your logic and the LLM.

### Behavior:
- Sends a `POST` request to `http://127.0.0.1:11434/v1/completions`
- Uses a simple prompt/response format
- Parses the result and returns the model’s response as plain text
- Includes basic error handling and a fallback agent name (`Planner`) if something fails


In [ ]:
def get_response(prompt):
    url = "http://127.0.0.1:11434/v1/completions"
    headers = {"Content-Type": "application/json"}
    data = {
        "prompt": prompt,
        "model": "deepseek-r1:7b"
    }

    try:
        response = requests.post(url, headers=headers, data=json.dumps(data))
        if response.status_code == 200:
            response_data = response.json()
            return response_data.get("choices", [{}])[0].get("text", "").strip()
        else:
            print(f"⚠️ Error: HTTP {response.status_code}")
            print(response.text)
            return "Planner"  # fallback to a valid agent name
    except Exception as e:
        print(f"❌ Request failed: {e}")
        return "PlannerBot"



## ⚙️ Choose Best Agent via LLM Prompt

This function constructs a natural language prompt describing a task and a list of agents, then uses a language model (`llm_func`) to decide which agent is the best fit.

### Key Concepts:
- **Agent Matching**: relies on each agent’s defined `goal` from `agent_specs`
- **Prompt Engineering**: creates a prompt that includes the task description, type, and complexity
- **LLM Decision**: expects the LLM to return only the exact name of the chosen agent
- **Post-processing**: strips and parses the raw LLM response for use in assignment logic


In [ ]:
def choose_best_agent(task, agents, llm_func):
    agent_descriptions = "\n".join([
    f"{i+1}. {name}: {spec['goal']}" for i, (name, spec) in enumerate(agent_specs.items())
])

    prompt = f"""
You are a task allocator AI.

Your job is to assign the following task to the most appropriate agent based on their expertise.

🔧 Task:
\"{task['description']}\" (Type: {task['type']}, Complexity: {task['complexity']})

🧠 Agents Available:
{agent_descriptions}

👉 Question:
Which agent is the best fit for this task? Respond only with the agent's **name**, exactly as written.
"""
    print(prompt)
    response = llm_func(prompt).split("</think>")[1]

    return response.strip()


## ⚙️ Generate Task Objects with LLM-Based Assignment

This loop transforms each raw task from `task_data` into a formal `Task` object and assigns it to the most suitable agent. The decision is made by querying the LLM via `choose_best_agent`.

### What happens:
- Each task is passed to the `choose_best_agent()` function
- The returned agent name is used to look up the corresponding agent instance
- A new `Task` object is created using `crewai.Task`
- The `description` and `expected_output` are passed through `dedent()` to clean up indentation
- Each task is printed with a confirmation message and stored in `task_objects`


In [ ]:
from crewai import Task
from textwrap import dedent

task_objects = []

for task in task_data:
    selected_name = choose_best_agent(task, agent_specs, get_response)
    print(selected_name)
    selected_agent = _agents[_agent_names.index(selected_name)]#.get(selected_name, list(agents.values())[0])  # fallback

    task_obj = Task(
        description=dedent(task["description"]),
        expected_output=dedent(f"Provide a full solution for: {task['description']}"),
        agent=selected_agent
    )
    task_objects.append(task_obj)
    print(f"✅ Assigned '{task['description']}' to {selected_name}")


## ⚙️ Create and Launch the Agent Crew

This block initializes the full multi-agent execution environment using the `CrewAI` framework. It ties together the agents and tasks, defines the coordination strategy, and executes the workflow.

### Components:
- `LLM(...)`: re-initializes the LLM interface for compatibility (can be reused or shared)
- `Crew(...)`: constructs the execution unit with:
  - `agents`: a list of previously instantiated agents
  - `tasks`: a list of assigned tasks
  - `process`: execution strategy (`sequential` or `parallel`)
- `crew.kickoff()`: begins the workflow, assigning each task to its agent
- `result`: stores the outcome of all tasks, which is printed after execution

In [ ]:
from crewai import Crew, Process
from crewai import LLM

llm = LLM(model="ollama/deepseek-r1:7b", base_url="http://127.0.0.1:11434")
crew = Crew(
    agents=_agents,
    tasks=task_objects,
    process=Process.sequential,


)
#print(list(agents.values()))
result = crew.kickoff()
print("🧠 Final Result:\n", result)



## 🖥️ Display Crew Results as Formatted Markdown

After the agents have completed their tasks, this block renders the results in a clean, readable Markdown format directly inside the notebook.

### Purpose:
- Uses `IPython.display.Markdown` to pretty-print structured output
- Assumes `result.raw` contains the full textual response from the `Crew` execution
- You can replace `result.raw` with `str(result)` or another attribute depending on how `CrewAI` returns output


In [ ]:
# @title 🖥️ Display the results of your crew as markdown
from IPython.display import display, Markdown

markdown_text = result.raw  # Adjust this based on the actual attribute

# Display the markdown content
display(Markdown(markdown_text))